# Lab 6

**Submission instructions**

Submit the Colab link to your notebook in Canvas. In addition, take screenshots of the map for each question and submit them to Canvas as well.

In [1]:
import ee
import geemap

In [2]:
geemap.ee_initialize()

## Question 1

Create a map to visualize [NOAA GFS Temperature Data](https://developers.google.com/earth-engine/datasets/catalog/NOAA_GFS0P25) and add a color bar and [NOAA logo](https://i.imgur.com/spILFEi.png) to the map. 

In [3]:
noaa_logo = "https://bit.ly/3ahJoMq"
made_by_text = "Made by Ilse Paniagua"

In [4]:
noaa_collection = (
    ee.ImageCollection('NOAA/GFS0P25')
    .filterDate('2022-01-01', '2022-12-31')
    .select('temperature_2m_above_ground')
    .median()
)



In [5]:
noaa_collection

In [6]:
temp_viz = {
    'palette': "hsv",
}

In [37]:
m = geemap.Map()

m.addLayer(noaa_collection, {})
m.add_colorbar(cmap="hsv", label="Temperature Above Ground")

m.add_text(made_by_text)
m.add_image(noaa_logo, position="bottomleft", image_size=(20, 20), xy=("2%", "80%"))
m

![](https://i.imgur.com/bBoRswR.png)

## Question 2

**Linked Maps:** Create a 2*2 linked map to visualize the Landsat imagery (`ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')`) with different band combinations.

In [8]:
landsat = ee.Image('LANDSAT/LE7_TOA_5YEAR/1999_2003')

In [9]:
landsat

In [10]:
vis_params = [
    {"bands": ["B3", "B2", "B1"], "gamma": 1.3},
    {"bands": ["B4", "B3", "B1"], "gamma": 1.3},
    {"bands": ["B2", "B3", "B4"], "gamma": 1.3},
    {"bands": ["B7", "B5", "B4"], "gamma": 1.3},
]

labels = [
    "Natural Color (B3/B2/B1)",
    "False Color (B4/B3/B1)",
    "Near Infrared (B2/B3/B4)",
    "Short Wave Infrared (B7/B5/B4)",
]

In [11]:
geemap.linked_maps(
    rows=2,
    cols=2,
    height="400px",
    center=[38.4151, 21.2712],
    zoom=12,
    ee_objects=[landsat],
    vis_params=vis_params,
    labels=labels,
    label_position="topright",
)

GridspecLayout(children=(Output(layout=Layout(grid_area='widget001')), Output(layout=Layout(grid_area='widget0…

![](https://i.imgur.com/Xp15uOo.png)

## Question 3

**Timeseries Inspector:** Create a map with timeseries inspector to visualize [USDA NASS Cropland Data Layers](https://developers.google.com/earth-engine/datasets/catalog/USDA_NASS_CDL) from 2010 to 2022. Add your name and [USDA logo](https://i.imgur.com/tzH2dNr.png) to the map.

In [12]:
usda_logo = "https://i.imgur.com/tzH2dNr.png"

In [13]:
cropland_collection = ee.ImageCollection("USDA/NASS/CDL").select("cropland").filterDate("2010-01-01", "2022-12-31")
years = cropland_collection.aggregate_array('system:index').getInfo()
years

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

In [14]:
p = geemap.Map(center=[40, -100], zoom=4)

p.add_layer(cropland_collection, {}, "NASS", False)
p.ts_inspector(
    left_ts=cropland_collection,
    right_ts=cropland_collection,
    left_name=labels,
    right_name=labels
)
p

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Dropdown(layout=…

![](https://i.imgur.com/gW1XPhL.png)

## Question 4

**Time slider:** Create a map with the time slider to visualize [Sentinel-2](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR) for Knoxville, TN.

In [28]:
tiger = ee.FeatureCollection("TIGER/2018/Counties")
washtenaw_filter =[ee.Filter.eq('STATEFP', '26'), ee.Filter.eq('NAME', 'Washtenaw')]
washtenaw = tiger.filter(washtenaw_filter)

In [29]:
sentinel = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
sentinel_clip = sentinel.filterBounds(washtenaw)\
                        .filter(ee.Filter.geometry(washtenaw)) \
                        .filterDate("2022-01-01", "2023-12-31") \
                        .filterMetadata("CLOUDY_PIXEL_PERCENTAGE", "less_than", 10) \
                        .limit(24)

In [30]:
sentinel_clip

Name,Description
B1,Aerosols
B2,Blue
B3,Green
B4,Red
B5,Red Edge 1
B6,Red Edge 2
B7,Red Edge 3
B8,NIR
B8A,Red Edge 4
B9,Water vapor


In [35]:
sentinel_viz = {
    "bands": ["B8", "B4", "B3"],
    "min": 0,
    "max": 4000,
    "gamma": [1.3, 1.3, 1.3],
    "opacity": 0.7
}

In [36]:
p = geemap.Map()
p.centerObject(washtenaw)
p.add_time_slider(
    sentinel_clip,
    sentinel_viz,
    time_interval=2)
p

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

![](https://i.imgur.com/rJe2rvP.png)

## Question 5

**Split-panel Map:** Use the following datasets to create a split-panel map for visualizing the ESA land cover data in the US. Add the ESA land cover legend to the map (Hint: the built-in legend for ESA land cover is `ESA_WorldCover`).

- [US Census States](https://developers.google.com/earth-engine/datasets/catalog/TIGER_2018_States): `ee.FeatureCollection("TIGER/2018/States")`
- [ESA WorldCover 10m](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100): `ee.ImageCollection("ESA/WorldCover/v100")`
- Landsat: `LANDSAT/LE7_TOA_5YEAR/1999_2003`


Currently, the split-map control of ipyleaflet plotting backend has a bug ([source](https://github.com/jupyter-widgets/ipyleaflet/issues/1066)). Use the folium plotting backend instead.

In [19]:
# Add your code here.

![](https://i.imgur.com/y0GW6k8.png)